In [ ]:
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs

%matplotlib inline

In [ ]:
# random sampole
!shuf -n 100000 /home/divgrad/data/4-Vadeboncoeur/davis-bay.txt > /home/divgrad/bcdata-project-temp/sample-temp.txt
data = pd.read_csv('/home/divgrad/bcdata-project-temp/sample-temp.txt',sep=" ", header=None)
!rm /home/divgrad/bcdata-project-temp/sample-temp.txt

From wiki:
<i>On the WGS84 spheroid, the length in meters of a degree of latitude at latitude</i> $\varphi$ is

$111132.92 - 559.82 \cos(2\varphi) + 1.175\cos(4\varphi ) - 0.0023 \cos(6\varphi)$,

... and <i>the length in meters of a degree of longitude<\i> is

$111412.84 \cos\varphi - 93.5\cos 3\varphi + 0.118 \cos 5\varphi $.

In [ ]:
column_names = ['lon', 'lat', 'z', 'r', 'g', 'b', '?', '?', '?']
data.columns = column_names



lon, lat = data['lon'].values, data['lat'].values

"""
min_lon, max_lon = dat_lon.min(), dat_lon.max()
min_lat, max_lat = dat_lat.min(), dat_lat.min()

lat_mid = 0.5*( min_lat + max_lat )

del_lat = 111132.92 - 559.82*np.cos(2*lat_mid) + 1.175*np.cos(4*lat_mid) - 0.0023*np.cos(6*lat_mid)
del_lon = 111412.84*np.cos(lat_mid) - 93.5*np.cos(3*lat_mid) + 0.118*np.cos(5*lat_mid)

x = (dat_lon - min_lon)*del_lon
y = (dat_lat - min_lat)*del_lat
"""

rgb = data.loc[:, ['r','g','b']].values
rgb_01 = rgb/255

plt.figure(figsize=(15,15))
plt.scatter(lon,lat,c=rgb_01,s=3,lw=0)
plt.grid()
plt.xlabel('longitude')
plt.ylabel('latitude')
plt.axis('scaled')

plt.show()

In [ ]:
print(rgb_01)

In [ ]:
from sklearn.neighbors import NearestNeighbors

lonlat = data.as_matrix(columns=data.columns)[:,:2]


In [ ]:
neigh = NearestNeighbors(radius=0.00001)
neigh.fit(lonlat)
rng = neigh.radius_neighbors([lonlat[0,:]])

print(np.asarray(rng[0][0]))
print(np.asarray(rng[1][0]))


In [ ]:
neigh = NearestNeighbors(radius=0.000005)
neigh.fit(lonlat)

# calculates graph showigng distnce between points if that distance is less than radius
A = neigh.radius_neighbors_graph(lonlat,mode='distance')

# calcualtes connectivity amtrix
#A = neigh.radius_neighbors_graph(lonlat,mode='connectivity')

In [ ]:
A.sum(axis=1).max()

In [ ]:
B = np.array([[1,1,1],[2,2,2]])

B.sum(axis=1)

In [ ]:
distance, indices = nbrs.kneighbors(lonlat)

In [ ]:
distance

In [ ]:
for k in range(5):
    print(lonlat[k,:])

In [ ]:
u_lonlat, u_ind = np.unique(lonlat,axis=0,return_index=True)

In [ ]:
a = np.array([[1, 0, 0], [1, 0, 0], [2, 3, 4]])
np.unique(a, axis=0)

## How good a representation is a random sample?

In [ ]:
num_samples = 5
num_bins = 'auto'

# min, max, mean, std
rgb_min = np.zeros((num_samples,3))
rgb_max = np.zeros((num_samples,3))
rgb_mean = np.zeros((num_samples,3))
rgb_std = np.zeros((num_samples,3))

plt.figure(figsize=(15,15))

for k in range(num_samples):

    !shuf -n 1000 /home/divgrad/data/4-Vadeboncoeur/davis-bay.txt > /home/divgrad/bcdata-project-temp/sample-temp.txt
    data = pd.read_csv('/home/divgrad/bcdata-project-temp/sample-temp.txt',sep=" ", header=None)
    !rm /home/divgrad/bcdata-project-temp/sample-temp.txt

    rgb = data.as_matrix(columns=data.columns)[:,2:5]
    
    rgb_min[k,:], rgb_max[k,:] = rgb.min(axis=0), rgb.max(axis=0)
    rgb_mean[k,:], rgb_std[k,:] = rgb.mean(axis=0), rgb.std(axis=0)
        
    plt.subplot(num_samples,3,3*k+1)
    plt.hist(rgb[:,0],color='r',bins=num_bins)
    plt.subplot(num_samples,3,3*k+2)
    plt.hist(rgb[:,1],color='g',bins=num_bins)
    plt.subplot(num_samples,3,3*k+3)
    plt.hist(rgb[:,2],color='b',bins=num_bins)

plt.show()


In [ ]:
# random sampole
!shuf -n 100000 /home/divgrad/data/4-Vadeboncoeur/davis-bay.txt > /home/divgrad/bcdata-project-temp/sample-temp.txt
data = pd.read_csv('/home/divgrad/bcdata-project-temp/sample-temp.txt',sep=" ", header=None)
!rm /home/divgrad/bcdata-project-temp/sample-temp.txt

data.columns = ['lon', 'lat', 'z', 'r', 'g', 'b', '?', '?', '?']

In [ ]:
scaler = StandardScaler()

X = data.as_matrix(columns=data.columns)[:,[0,1,3,4,5]]
X = StandardScaler().fit_transform(X)

In [ ]:
db = DBSCAN(eps=0.1, min_samples=250).fit(X)

In [ ]:
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)


# #############################################################################
# Plot result
import matplotlib.pyplot as plt

# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]

plt.figure(figsize=(15,15))
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = (labels == k)

    xy = X[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='none', markersize=14)

    xy = X[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='none', markersize=6)

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()

In [ ]:
print('Estimated number of clusters: %d' % n_clusters_)